**About Titanic**
=================

RMS Titanic, during her maiden voyage on April 15, 1912, sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. The tragedy is considered one of the most infamous shipwrecks in history and led to better safety guidelines for ships.

**I have approached this project through the following process:**

 1. Understand different features in the training dataset
 2. Clean the features
 3. Remove outliers
 4. Find relation between different features and survival 
 5. Find the best features using SelectKBest (to get an optimal fit between bias and variance)
 6. Train and fit the model
 7. Predict the scores using KNearestNeighbors
 8. Check Accuracy
 9. Predict Survival values for test.csv
 10. Create final file for submission



In [78]:
# Importing related Python libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import csv

# Importing SKLearn clssifiers and libraries
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV



In [79]:
# Importing the training dataset

df_train = pd.read_csv('train.csv')

#  1. Understand different features in the training dataset

The training dataset, imported as a Pandas dataframe (train_df) has 891 rows with 12 columns/features, with some of the details mentioned below: 

In [80]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [81]:
df_train.shape

(891, 12)

In [82]:
df_train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

# 2. Clean the features

I am not interested the following features, as I believe they do not hold any logical reason, why the passenger will survive or not :

 1. Name
 2. Ticket 
 3. Fare 
 4. Cabin

Further, in case I find correlation between them, I believe it will not be causation (Ticket, Fare, and Cabin can be highly correlated to Passenger Class)

Therefore, I have dropped the above-mentioned features from the training dataset


In [83]:
df_train['Family'] = df_train['SibSp'] + df_train['Parch'] + 1

I have created another column/feature for number of people in the family (Family) by adding SibSp, Parch, and 1 (for the passenger). 

Further, I have dropped the SibSp and Parch features



In [84]:
df_train = df_train.drop('SibSp', axis=1,)
df_train = df_train.drop('Parch', axis=1,)

I also want to check a hypothesis that while saving passengers minors were given preference over adults. 

Therefore, I would be creating a new column to differentiate minors from adults. However, I will first check the Age statistics of passengers.

In [85]:
df_train["Age"].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

Since the column Age contains several NaN values, I will first clear those values in next section, and then create the new column.

#  3. Remove outliers

Now, I want to check which features have how many NaN cells

In [86]:
feat_list = list(df_train.columns.values)

for feat in feat_list:
    print (feat,": ",sum(pd.isnull(df_train[feat])))

PassengerId :  0
Survived :  0
Pclass :  0
Name :  0
Sex :  0
Age :  177
Ticket :  0
Fare :  0
Cabin :  687
Embarked :  2
Family :  0


We have 177 NaN in Age and 2 in Embarked.

 1. For Age, I have filled the NaN with median age
 2. For Embarked, I have filled the NaN with port of embarkation with maximum frequency

In [87]:
df_train["Age"] = df_train["Age"].fillna(df_train["Age"].median())
df_train["Embarked"].mode()
df_train["Embarked"] = df_train["Embarked"].fillna("S")

In [88]:
# re-checking NaNs

feat_list = list(df_train.columns.values)

for feat in feat_list:
    print (feat,": ",sum(pd.isnull(df_train[feat])))

PassengerId :  0
Survived :  0
Pclass :  0
Name :  0
Sex :  0
Age :  0
Ticket :  0
Fare :  0
Cabin :  687
Embarked :  0
Family :  0


In [89]:
# Checking statistics of Age column
df_train["Age"].describe()

count    891.000000
mean      29.361582
std       13.019697
min        0.420000
25%       22.000000
50%       28.000000
75%       35.000000
max       80.000000
Name: Age, dtype: float64

As discussed before, I have now categorized the Age feature into following two categories:

 1. Passengers who were minors (0)
 2. passengers who were adults (1)

I have created a new column named Adult, and assigned value 1 to passengers who were 18 years and above, and 0 to passengers who were minors. 

This is done to check the hypothesis that more minors survived, as compared to adults

In [90]:
df_train["Adult"] = 0

In [91]:
df_train["Adult"][df_train["Age"] >= 18] = 1

/var/folders/bm/211br2s556ngq8mdsy98h_j40000gn/T/ipykernel_2976/1201628888.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["Adult"][df_train["Age"] >= 18] = 1


In [93]:
### Dropping the Age column

df_train = df_train.drop('Age', axis=1,)

In [121]:
df_train.head()

,PassengerId,Survived,Pclass,Sex,Embarked,Family,Adult
0,1,0,3,male,S,2,1
1,2,1,1,female,C,2,1
2,3,1,3,female,S,1,1
3,4,1,1,female,S,2,1
4,5,0,3,male,S,1,1


In [95]:
df_train = df_train.drop('Name', axis=1,)
df_train = df_train.drop('Ticket', axis=1,)
df_train = df_train.drop('Fare', axis=1,)
df_train = df_train.drop('Cabin', axis=1,)

In [96]:
df_train.dtypes

PassengerId     int64
Survived        int64
Pclass          int64
Sex            object
Embarked       object
Family          int64
Adult           int64
dtype: object

#  5. Find the best features using SelectKBest (to get an optimal fit between bias and variance)

From the df_train dataframe, I have created a dataframe X, which contains all the features, and a numpy array y, which contains values of survived passengers

In [97]:
df1 = df_train.filter(['Pclass','Sex','Embarked','Family','Adult'], axis=1)

X = df1

In [98]:
df2 = df_train['Survived']

y = df2

in order to run SelectKBest, I have converted values in "Embarked"  and "Sex" to floats

In [99]:
X["Embarked"].unique()

array(['S', 'C', 'Q'], dtype=object)

In [100]:
X["Embarked"][df_train["Embarked"] == "S"] = 1
X["Embarked"][df_train["Embarked"] == "C"] = 2
X["Embarked"][df_train["Embarked"] == "Q"] = 3

/var/folders/bm/211br2s556ngq8mdsy98h_j40000gn/T/ipykernel_2976/4178034214.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Embarked"][df_train["Embarked"] == "S"] = 1
/var/folders/bm/211br2s556ngq8mdsy98h_j40000gn/T/ipykernel_2976/4178034214.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Embarked"][df_train["Embarked"] == "C"] = 2
/var/folders/bm/211br2s556ngq8mdsy98h_j40000gn/T/ipykernel_2976/4178034214.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [101]:
X["Sex"][df_train["Sex"] == "male"] = 1
X["Sex"][df_train["Sex"] == "female"] = 2


/var/folders/bm/211br2s556ngq8mdsy98h_j40000gn/T/ipykernel_2976/3711890699.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Sex"][df_train["Sex"] == "male"] = 1
/var/folders/bm/211br2s556ngq8mdsy98h_j40000gn/T/ipykernel_2976/3711890699.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Sex"][df_train["Sex"] == "female"] = 2


In [102]:
# using SelectKBest to get scores of all features of the DataFrame

test = SelectKBest(f_classif, k='all')
test_fit = test.fit(X, y)
feat_score = test_fit.scores_.round(3)
p_values = -np.log10(test_fit.pvalues_).round(3)

In [103]:
feature_list = list(X.columns.values)
selected_features = test.get_support([test_fit])
selected_features

array([0, 1, 2, 3, 4])

In [104]:
temp_list = [ ]

for i in selected_features:
    temp_list.append({'Feature':feature_list[i], 'P_Value':p_values[i], 'Score': feat_score[i]  })
    
feat_select = pd.DataFrame(temp_list)

In [105]:
feat_select = feat_select.sort_values(by='Score', axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last')

In [106]:
feat_select = feat_select.set_index('Feature')

In [107]:
feat_select

,P_Value,Score
Feature,,
Sex,68.852,372.406
Pclass,24.596,115.031
Adult,3.594,13.485
Embarked,2.851,10.259
Family,0.208,0.246


**Based on the score above, I have considered top-3 features - Sex, Pclass and Adult - as my final features**

In [108]:
### Dropping the Embarked and Family column

X = X.drop('Embarked', axis=1,)
X = X.drop('Family', axis=1,)


In [109]:
X.head()

,Pclass,Sex,Adult
0,3,1,1
1,1,2,1
2,3,2,1
3,1,2,1
4,3,1,1


#  6. Train and fit the model

Since the test.csv does not have "Survived" column, I have split the training dataset to do in-house accuracy and precision testing, before submitting the entry 

In [110]:
features_train, features_test, labels_train, labels_test = \
    train_test_split(X, y, test_size=0.3, random_state=42)

In [111]:
features_train.shape

(623, 3)

In [112]:
features_test.shape

(268, 3)

In [113]:
labels_train.shape

(623,)

In [114]:
labels_test.shape

(268,)

# 7. Predict the scores using KNearestNeighbors

**Using K Nearest Neighbors (KNN) with GridSearchCV**

In [115]:
knn = KNeighborsClassifier( )
k_range = list(range(1,10))
weights_options = ['uniform','distance']
k_grid = dict(n_neighbors=k_range, weights = weights_options)
grid = GridSearchCV(knn, k_grid, cv=10, scoring = 'precision')
grid.fit(features_train, labels_train)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'weights': ['uniform', 'distance']},
             scoring='precision')

In [116]:
print ("Best Score: ",str(grid.best_score_))

Best Score:  0.9233022533022532


In [117]:
print ("Best Parameters: ",str(grid.best_params_))

Best Parameters:  {'n_neighbors': 4, 'weights': 'distance'}


In [118]:
print ("Best Estimators: ",str(grid.best_estimator_))

Best Estimators:  KNeighborsClassifier(n_neighbors=4, weights='distance')


#  8. Check Accuracy

In [119]:
# predicting scores

label_pred = grid.predict(features_test)

In [120]:
# Calculating Accuracy

acc_clf = metrics.accuracy_score(labels_test,label_pred)
print ("classifier's accuracy: ",str(acc_clf) )

classifier's accuracy:  0.7835820895522388
